# Imports and Data loading

In [46]:
######### List of imports ################
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from matplotlib.pyplot import figure
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.graphics.tsaplots import plot_predict
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf
import pmdarima as pm
from ThymeBoost import ThymeBoost as tb

###########################################

In [47]:
## Read data from "BTCUSD"

data = pd.read_csv("/Users/yuhaoli/code/HowardLiYH/Volatility-prediction/datasets/raw/data0329.csv")

# Feature Creation

In [48]:
data.head()

,bp_1,bz_1,bp_2,bz_2,bp_3,bz_3,bp_4,bz_4,bp_5,bz_5,...,ap_17,az_17,ap_18,az_18,ap_19,az_19,ap_20,az_20,scrape_time,lastUpdateId
0,27215.98,3.62823,27215.71,0.09094,27215.15,0.06106,27215.14,0.10176,27213.06,0.11000,...,27220.22,0.18378,27220.24,0.18378,27220.40,0.38577,27220.87,0.73017,2023-03-28 19:33:30,17688482161
1,27219.01,2.49441,27218.96,0.40455,27218.95,0.81592,27218.94,0.56887,27218.86,0.51754,...,27221.64,0.05013,27221.79,0.03673,27222.27,0.04246,27222.44,0.03673,2023-03-28 19:33:24,17688480766
2,27219.01,0.53591,27218.72,0.41551,27218.71,0.07000,27218.40,0.00671,27218.36,0.57475,...,27223.09,0.22057,27223.10,0.36755,27223.24,0.46670,27223.25,0.00367,2023-03-28 19:33:19,17688480114
3,27223.23,6.40427,27221.59,1.00260,27221.39,0.03674,27221.36,1.10536,27221.34,1.10536,...,27227.34,0.18374,27227.92,1.06032,27228.39,1.21947,27228.40,2.03245,2023-03-28 19:33:36,17688483548
4,27223.23,0.48052,27222.67,0.48829,27222.40,0.09468,27222.29,0.57712,27222.22,0.00921,...,27227.34,0.36748,27228.05,0.03622,27228.07,1.01425,27228.08,0.78360,2023-03-28 19:33:07,17688478201


In [49]:
# Weighted Average Price 
data['WAP'] = (data['bp_1']*data['bz_1']
               +data['bp_2']*data['bz_2']
               +data['ap_1']*data['az_1']
               +data['ap_2']*data['az_2'])/(data['bz_1']+
                                         data['bz_2']+
                                         data['az_1']+
                                         data['az_2'])

In [50]:
## Spread
data['spread'] = ((data['ap_1']/data['bp_1']) - 1)

In [51]:
## Log price

def log_price(list_stock_prices):
    return np.log(list_stock_prices)

data.insert(0, 'log_price', log_price(data['WAP']))

data['log_returns'] = data.log_price.diff()

In [52]:
## Realized Volatility
def realized_volatility():
    list_vol = []
    i = 0
    for i in data.index:
        x = np.std(data.log_returns.iloc[:i])
        i += 1
        list_vol.append(x)
    
    return list_vol

data['realized_volatility'] = realized_volatility()

In [53]:
## Previous volatility

data['volatility_t+1'] = data['realized_volatility'].shift(-1)

In [54]:
y = data['realized_volatility'].to_frame()

In [55]:
## Create a copy of data named df
df = data

In [56]:
## Drop the target y
df.drop(['realized_volatility'], axis = 1, inplace = True) 

In [57]:
## Check if two roles have the same value
df.duplicated().value_counts()

False    2644
dtype: int64

# Deal with Missing Data

In [58]:
imputer = SimpleImputer(strategy="constant", fill_value = 0) # Instantiate a SimpleImputer object with your strategy of choice

imputer.fit(df[['volatility_t+1']]) # Call the "fit" method on the object

df['volatility_t+1'] = imputer.transform(df[['volatility_t+1']]) # Call the "transform" method on the object

imputer.statistics_ # The mean is stored in the transformer's memory

array([0.])

In [59]:
imputer2 = SimpleImputer(strategy="constant", fill_value = df.iloc[1,85])

imputer2.fit(df[['log_returns']]) # Call the "fit" method on the object

df['log_returns'] = imputer2.transform(df[['log_returns']]) # Call the "transform" method on the object

imputer2.statistics_ # The mean is stored in the transformer's memory

array([0.00011149])

In [60]:
df.columns

Index(['log_price', 'bp_1', 'bz_1', 'bp_2', 'bz_2', 'bp_3', 'bz_3', 'bp_4',
       'bz_4', 'bp_5', 'bz_5', 'bp_6', 'bz_6', 'bp_7', 'bz_7', 'bp_8', 'bz_8',
       'bp_9', 'bz_9', 'bp_10', 'bz_10', 'bp_11', 'bz_11', 'bp_12', 'bz_12',
       'bp_13', 'bz_13', 'bp_14', 'bz_14', 'bp_15', 'bz_15', 'bp_16', 'bz_16',
       'bp_17', 'bz_17', 'bp_18', 'bz_18', 'bp_19', 'bz_19', 'bp_20', 'bz_20',
       'ap_1', 'az_1', 'ap_2', 'az_2', 'ap_3', 'az_3', 'ap_4', 'az_4', 'ap_5',
       'az_5', 'ap_6', 'az_6', 'ap_7', 'az_7', 'ap_8', 'az_8', 'ap_9', 'az_9',
       'ap_10', 'az_10', 'ap_11', 'az_11', 'ap_12', 'az_12', 'ap_13', 'az_13',
       'ap_14', 'az_14', 'ap_15', 'az_15', 'ap_16', 'az_16', 'ap_17', 'az_17',
       'ap_18', 'az_18', 'ap_19', 'az_19', 'ap_20', 'az_20', 'scrape_time',
       'lastUpdateId', 'WAP', 'spread', 'log_returns', 'volatility_t+1'],
      dtype='object')

In [61]:
#sum of all bid quantities
data['bid depth'] = data[['bz_1', 'bz_2', 'bz_3','bz_4', 'bz_5', 'bz_6','bz_7', 'bz_8', 'bz_9','bz_10',
                         'bz_11', 'bz_12', 'bz_13','bz_14', 'bz_15', 'bz_16','bz_17', 'bz_18', 'bz_19','bz_20']].sum(axis=1)

In [62]:
#sum of all bid quantities
data['ask depth'] = data[['az_1', 'az_2', 'az_3','az_4', 'az_5', 'az_6','az_7', 'az_8', 'az_9','az_10',
                         'az_11', 'az_12', 'az_13','az_14', 'az_15', 'az_16','az_17', 'az_18', 'az_19','az_20']].sum(axis=1)

In [63]:
#Order Flow Imbalance (OFI) 
#relative quantities of bids vs asks
#full depth (approx 20 levels) OFI
data['FDOFI'] = (data['bid depth']-data['ask depth'])/(data['bid depth']+data['ask depth'])

In [64]:
df = df.set_index('scrape_time')

In [65]:
imputer3 = SimpleImputer(strategy="constant", fill_value = 0) # Instantiate a SimpleImputer object with your strategy of choice

imputer3.fit(y[['realized_volatility']]) # Call the "fit" method on the object

y['realized_volatility'] = imputer3.transform(y[['realized_volatility']]) # Call the "transform" method on the object

imputer3.statistics_ # The mean is stored in the transformer's memory

array([0.])

# Boosted-Linear Auto-ARIMA (mape:0.0396) 

In [66]:
train = y[0:1720]['realized_volatility']
test = y[1720:]['realized_volatility']

In [67]:
boosted_model = tb.ThymeBoost(verbose=1)
output = boosted_model.fit(train,
                            trend_estimator=['linear', 'arima'],
                            arima_order='auto',
                            global_cost='mse')
predicted_output = boosted_model.predict(output, len(test))

********** Round 1 **********
Using Split: None
Fitting initial trend globally with trend model:
median()
seasonal model:
None
cost: 1.7084236832135604e-07
********** Round 2 **********
Using Split: None
Fitting global with trend model:
linear((1, None))
seasonal model:
None
cost: 1.706560104552915e-07
********** Round 3 **********
Using Split: None
Fitting global with trend model:
arima(auto)
seasonal model:
None
cost: 2.0971056201402757e-09
Boosting Terminated 
Using round 3


In [68]:
tb_mape, tb_mae, tb_rmse

(0.03956069325021652, 4.423218013320679e-05, 5.6293469389802835e-05)

# Model Testing 

In [69]:
tb_mae = np.mean(np.abs(test - predicted_output['predictions']))
tb_rmse = (np.mean((test - predicted_output['predictions'])**2))**.5
tb_mape = np.sum(np.abs(predicted_output['predictions'] - test)) / (np.sum((np.abs(test))))

tb_mape, tb_mae, tb_rmse

(0.03956069325021652, 4.423218013320679e-05, 5.6293469389802835e-05)